In [ ]:
pip install scikit-fuzzy

In [ ]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

Generem dades artificials que de moment no tenim les oficials

In [17]:
import pandas as pd
import numpy as np

# Llista dels 51 estats dels EUA (noms complets)
estats_usa = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware",
    "District of Columbia", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa",
    "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota",
    "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico",
    "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island",
    "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington",
    "West Virginia", "Wisconsin", "Wyoming"
]
# Genera dades aleatòries realistes (ajustables)
np.random.seed(42)  # Per reprodibilitat

# Genera valors aleatoris per hospitalitzats, morts i contagiats
df_epidemiologic = pd.DataFrame({
    "state": estats_usa,
    "hospitalitzats": np.random.randint(50, 2000, size=51),  # Exemple: entre 50 i 2000
    "morts": np.random.randint(5, 500, size=51),            # Exemple: entre 5 i 500
    "contagiats": np.random.randint(1000, 50000, size=51)   # Exemple: entre 1000 i 50000
})

# Mostra les primeres files
print(df_epidemiologic.head())

        state  hospitalitzats  morts  contagiats
0     Alabama            1176     55       34763
1      Alaska            1509    368       33606
2     Arizona             910     59       12534
3    Arkansas            1344    248       30127
4  California            1180    324       41397


In [18]:
import pandas as pd

df_demografic_original = pd.read_csv("../data/preprocessed/dataMatrix/static_stateMatrix.csv")
columnes = [
    "state", 
    "population_state",
    "pop_density_state",
    "bedsTotal",
    "pop_0-9", "pop_10-19", "pop_20-29", "pop_30-39",
    "pop_40-49", "pop_50-59", "pop_60-69", "pop_70-79", "pop_80+"
]
df_demografic = df_demografic_original[columnes]

Fusionem per tenir dades població (x edad ens poden servir també x tenir zones amb molta gent gran)

In [19]:
# Fusiona els dos DataFrames per la columna "state"
df_combinat = pd.merge(
    df_demografic, 
    df_epidemiologic, 
    on="state", 
    how="inner" 
)

# Mostra les primeres files (opcional)
print(df_combinat.head())

        state  population_state  pop_density_state  bedsTotal   pop_0-9  \
0     Alabama           4887871          96.509389        3.1  0.120933   
1      Alaska            737438           1.291523        2.2  0.142617   
2     Arizona           7171646          63.135855        1.9  0.123774   
3    Arkansas           3013825          57.919684        3.2  0.127289   
4  California          39557045         253.906502        1.8  0.124635   

   pop_10-19  pop_20-29  pop_30-39  pop_40-49  pop_50-59  pop_60-69  \
0   0.128108   0.136099   0.124291   0.123013   0.133249   0.120537   
1   0.130210   0.155501   0.147626   0.117347   0.127204   0.109072   
2   0.132496   0.140854   0.129224   0.119039   0.120016   0.113492   
3   0.132222   0.135421   0.126671   0.119787   0.126968   0.115755   
4   0.128760   0.147575   0.143412   0.128138   0.126974   0.104040   

   pop_70-79   pop_80+  hospitalitzats  morts  contagiats  
0   0.075243  0.038527            1176     55       34763  
1 

In [20]:
# IA (Incidència Acumulada)
df_combinat["IA"] = (df_combinat["contagiats"] / df_combinat["population_state"]) * 100000

# θ (Ocupació hospitalària %)
df_combinat["beds_real"] = (df_combinat["bedsTotal"] * df_combinat["population_state"]) / 1000
df_combinat["theta"] = (df_combinat["hospitalitzats"] / df_combinat["beds_real"]) * 100

# π (Taxa de mortalitat %) per 100000 habitants
df_combinat["pi"] = (df_combinat["morts"] / df_combinat["population_state"]) * 100000

# % població >65
df_combinat["pop_>65"] = df_combinat["pop_70-79"] + df_combinat["pop_80+"]


In [21]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# Definir variables d'entrada i sortida
ia = ctrl.Antecedent(np.arange(0, 1001, 1), 'ia')                  # Incidència acumulada (per 100k)
ocupacio = ctrl.Antecedent(np.arange(0, 101, 1), 'ocupacio')       # Ocupació hospitalària (%)
mortalitat = ctrl.Antecedent(np.arange(0, 101, 1), 'mortalitat')   # Taxa de mortalitat (per 100k)
poblacio65 = ctrl.Antecedent(np.arange(0, 101, 1), 'poblacio65')   # % població >65 anys
densitat = ctrl.Antecedent(np.arange(0, 10001, 1), 'densitat')     # Densitat (hab/km²)
risc = ctrl.Consequent(np.arange(0, 101, 1), 'risc')               # Risc de confinament (0-100)

# Funcions de membresia per a cada variable (ajustar rangs segons els teus datos)
# --- IA ---
ia['Baixa'] = fuzz.trimf(ia.universe, [0, 0, 300])
ia['Mitjana'] = fuzz.trimf(ia.universe, [200, 400, 600])
ia['Alta'] = fuzz.trimf(ia.universe, [500, 1000, 1000])

# --- Ocupació Hospitalària ---
ocupacio['Baixa'] = fuzz.trimf(ocupacio.universe, [0, 0, 30])
ocupacio['Moderada'] = fuzz.trimf(ocupacio.universe, [20, 50, 80])
ocupacio['Crítica'] = fuzz.trimf(ocupacio.universe, [60, 100, 100])

# --- Mortalitat ---
mortalitat['Baixa'] = fuzz.trimf(mortalitat.universe, [0, 0, 30])
mortalitat['Mitjana'] = fuzz.trimf(mortalitat.universe, [20, 50, 80])
mortalitat['Alta'] = fuzz.trimf(mortalitat.universe, [60, 100, 100])

# --- Població >65 ---
poblacio65['Baixa'] = fuzz.trimf(poblacio65.universe, [0, 0, 20])
poblacio65['Mitjana'] = fuzz.trimf(poblacio65.universe, [15, 30, 45])
poblacio65['Alta'] = fuzz.trimf(poblacio65.universe, [40, 100, 100])

# --- Densitat ---
densitat['Baixa'] = fuzz.trimf(densitat.universe, [0, 0, 2000])
densitat['Mitjana'] = fuzz.trimf(densitat.universe, [1500, 4000, 6000])
densitat['Alta'] = fuzz.trimf(densitat.universe, [5000, 10000, 10000])

# --- Risc ---
risc['Baix'] = fuzz.trimf(risc.universe, [0, 0, 30])
risc['Moderat'] = fuzz.trimf(risc.universe, [20, 50, 80])
risc['Alt'] = fuzz.trimf(risc.universe, [60, 80, 100])
risc['Extrem'] = fuzz.trimf(risc.universe, [80, 100, 100])

In [22]:
# Definir regles basades en combinacions de variables (exemples):
rule1 = ctrl.Rule(
    ia['Alta'] & ocupacio['Crítica'] & mortalitat['Alta'],
    risc['Extrem']
)
rule2 = ctrl.Rule(
    ia['Mitjana'] & ocupacio['Moderada'] & (poblacio65['Alta'] | densitat['Alta']),
    risc['Alt']
)
rule3 = ctrl.Rule(
    ia['Baixa'] & ocupacio['Baixa'] & mortalitat['Baixa'],
    risc['Baix']
)
rule4 = ctrl.Rule(
    (ia['Mitjana'] | mortalitat['Mitjana']) & densitat['Mitjana'],
    risc['Moderat']
)
rule5 = ctrl.Rule(
    ocupacio['Crítica'] & (poblacio65['Alta'] | mortalitat['Alta']),
    risc['Alt']
)

# Crear sistema de control
risc_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5])
risc_sim = ctrl.ControlSystemSimulation(risc_ctrl)

In [27]:
# Aplicar a cada fila del DataFrame
def calcular_risc(row):
    try:
        # Definir el sistema de control cada vegada (evita problemes d'estat)
        risc_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5])
        risc_sim = ctrl.ControlSystemSimulation(risc_ctrl)
        
        # Assignar valors (noms exactes!)
        risc_sim.input['ia'] = row['IA']
        risc_sim.input['ocupacio'] = row['theta']
        risc_sim.input['mortalitat'] = row['pi']
        risc_sim.input['poblacio65'] = row['pop_>65']
        risc_sim.input['densitat'] = row['pop_density_state']
        
        risc_sim.compute()
        return risc_sim.output['risc']
    except Exception as e:
        print(f"Error a la fila {row.name}: {e}")
        return 0  # Valor per defecte

df_combinat['nivell_risc'] = df_combinat.apply(calcular_risc, axis=1)
df_combinat['risc'] = df_combinat.apply(calcular_risc, axis=1)

# Classificar decisió de confinament
df_combinat['confinar'] = np.where(
    df_combinat['nivell_risc'] >= 70,  # Ajusta el llindar
    'Sí',
    'No'
)

Error a la fila 0: 'risc'
Error a la fila 1: 'risc'
Error a la fila 3: 'risc'
Error a la fila 5: 'risc'
Error a la fila 7: 'risc'
Error a la fila 8: 'risc'
Error a la fila 11: 'risc'
Error a la fila 12: 'risc'
Error a la fila 14: 'risc'
Error a la fila 15: 'risc'
Error a la fila 16: 'risc'
Error a la fila 17: 'risc'
Error a la fila 19: 'risc'
Error a la fila 20: 'risc'
Error a la fila 21: 'risc'
Error a la fila 22: 'risc'
Error a la fila 26: 'risc'
Error a la fila 27: 'risc'
Error a la fila 28: 'risc'
Error a la fila 29: 'risc'
Error a la fila 30: 'risc'
Error a la fila 31: 'risc'
Error a la fila 33: 'risc'
Error a la fila 34: 'risc'
Error a la fila 36: 'risc'
Error a la fila 37: 'risc'
Error a la fila 39: 'risc'
Error a la fila 41: 'risc'
Error a la fila 42: 'risc'
Error a la fila 44: 'risc'
Error a la fila 46: 'risc'
Error a la fila 48: 'risc'
Error a la fila 49: 'risc'
Error a la fila 50: 'risc'
Error a la fila 0: 'risc'
Error a la fila 1: 'risc'
Error a la fila 3: 'risc'
Error a la

In [ ]:
df_combinat.head()


In [ ]:
df_demografic_original.columns

In [ ]:
df_demografic.columns

In [ ]:
df_combinat.columns

In [ ]:
df_combinat.head()

## Aplicació de regles de coneixement difús 

In [ ]:

def classifica_risc(row):
    if row["Rt"] > 1.5 and row["IA"] > 150 and row["theta"] > 80:
        return "Crític"
    elif row["Rt"] > 1.2 and (row["IA"] > 100 or row["pi"] > 1.5) and row["theta"] > 70:
        return "Alt"
    elif row["IA"] > 50 and row["positivity_rate"] > 5:
        return "Moderat"
    else:
        return "Baix"

df_combinat["nivell_risc"] = df_combinat.apply(classifica_risc, axis=1)

# Mostrem les dades clau resultants
print(df_combinat[["state", "IA", "Rt", "theta", "pi", "positivity_rate", "pop_>65", "nivell_risc"]].head())


In [ ]:
df_combinat.head()

-----------------------------------------